## RNAseq in a nutshell: From FASTQ files to differential expression

While expression arrays have been widely used (historically) to quantify transcript abundance, high throughput sequencing is extremely heavily utilized today instead, for its many advantages.

We will start this session by describing the structure and quality control measures used for fastq files. So, it is important that before you start the next two sessions, you understand how these files are generated as part of an RNAseq experimental pipeline. This is composed of two steps: library generation and high throughput sequencing. 

**Library generation:** RNA extracted from cells, model organism or tissue undergoes a procedure that converts the RNA into DNA while attempting to preserve the relative abundance of the different transcripts. It is important to remember that the total RNA in a cell is usually composed of mostly rRNA, which is usually depleted in the process of library prep.

**Q1.** Take a look at figure 1a in the paper by Martin et al. which we provided for you on Canvas, when going from step 3 to 4 primers need to be added for the reverse transcriptase, usually two types are used, which primers would you used if you would like to make sure that you are sequencing only mRNA? (If you have never heard of cDNA before, look it up using google)

**High throughput sequencing:** Many technologies for massively parallel sequencing have been introduced over the years but definitely the most popular is the Illumina Sequencing-By-Synthesis (SBS) technology. The DNA fragments that undergo SBS are the ones depicted in step 7 in figure 1a mentioned above. 

**Q2.** Take a look at an introduction video by <a href="https://www.youtube.com/watch?v=fCd6B5HRaZ8&ab_channel=Illumina" target="_blank">Illumina on SBS</a>, Illumina sequencing has limitations on the size of the fragments that can be sequenced, too short or too long DNA fragments cannot be accurately sequenced. Based on this video, can you guess which step in the process is responsible for this limitation?

Note that sequencing can be done using either single or paired-end reads, as the fragments are usually longer than the number of SBS cycles, a paired-end read would capture the full sequence of the fragment by sequencing the two ends and filling the middle part using the genomic of transcriptome reference sequence.



### Structure of the FASTQ file

Illumina sequencing reads are stored in FASTQ files, in these files every read is stored as four rows as follows:

* Line 1:	Sequence ID – will always start with @ and is a unique ID for the read, this line is important because in many cases each sequencing cluster will have more than one read (like paired end and sample barcode) which will need to be matched.
* Line 2: Sequence – The actual sequence consisting of the four nucleotides and N in cases where the precise nucleotide could not have been determined.
* Line 3: Quality ID – This line will always start with + which in some cases will be followed by and read quality score encoded in ASCII
* Line 4: Quality score – Note that this line will contain the same number of characters as the sequence line, every letter encodes the quality of the corresponding nucleotide in the sequence


**Q3.** We have provided an example FASTQ file in the "Data" folder assigned with this module (and inside, the file: `sample.fastq`). Using the cocalc interface, open this file and look at the top lines of the file. What is the nucleotide sequence of the 10th read? How many reads total does this file contain? (Remember that each read is four lines!)

### Assessing FASTQ files quality and sequence composition with FASTQC 

FASTQC is an extremely convenient tool that you should use as your first step in testing the quality of an Illumina NGS run. FASTQC will analyze the nucleotide composition and sequence quality scores of your run and will output and html file for each `fastq` file summarizing the results. 

We have provided you a **very** small snippet of the full `fastq` file. Let's run it on this small bit of data. 

We haven't used UNIX in the course (yet!), but we will soon. UNIX is a command line environment where you can run program to analyze data, parse files, and generally where you will keep and your computational analytical pipelines.

Without worrying *too* much about this, we can run the program `fastqc` on the UNIX command line in the following way:

- In your notebook, click on the "Jupyter" menu button, select "[+] New Frame", then ">_ Terminal" from the drop down menu. You should then see a UNIX command line prompt that looks like this:

     `~/07_RNASeq-I$` 


- The part before the dollar sign tells you what folder (or in UNIX, 'directory') you are currently working in. We'll try to refer to `$>` as the "prompt", such that when we ask you to execute a command in UNIX, you'll type in everything *after* the prompt.


- Yes! This is your first foray into working in the UNIX command environment. Don't be scared! In just a few modules from now, you'll get a deeper dive into this framework.

For now, follow along for practice to get used to what this will be like.

- The first thing we need to do is create a brand new folder to saved some temporary files. To do that, execute the following command:

    `$> mkdir ~/tmp`
    

- This will make a temporary folder (called a directory in UNIX; here, "make directory" is abbreviated as "mkdir".) 
- Then, execute the following command:

    `$> fastqc -o . -d ~/tmp ~/07_Data_RNASeq-I/sample.fastq`
       
In general, this call the program "fastqc" which will analyze the file `sample.fastq` located in the Data directory (we shared this directory along with this module), and will save the output (`-o`) in your file in your current directory. The program creates temporary files, and so we tell the program what directory it should use for that (`-d`; and use the directory you just made, above).

**Q4.** Open the resulting `sample_fastqc.html` file that was created in your directory in cocalc. How many reads were analyzed in this sample file?

The full `.fastq` file for these data is quite large (~24Gb!).

So we have provided an example `fastqc` output file with the result of this tool run on the full data (`full_fastqc.html`).

Open this file (you might need to download it to your computer, which you can do from the file explorer in cocalc if you need to) and explore the different summary statistics. Take a look at the per base sequence content, note the difference in nucleotide distribution at the begging of the reads compared to the middle of the reads. This is also reflected in the k-mer count. 

**Q5.** What is the second most overrepresented k-mer? 

**Q6.** What possible sources in the process of data generation could generate these k-mer count biases? (Hint: Think about library prep.)

### Mapping of reads to the genome/transcriptome

The next step is to map the reads, representing RNA fragments, to a reference sequence library (as the genome) in order to convert the `.fastq` files containing DNA fragments to a gene-based value associated with expression levels.

We should note here that in most cases, you will have ligated adaptors/barcodes in the course of library preparations, so that you can 'pool' case/controls samples together into a single run of sequencing. This is good experimental design, so that (i) cases and controls are sequencing in the same way so that you minimize batch effects, and (ii) because sequencing machines generate *so much data*, you don't need a single biological sample to carry a zillion reads, so you can sequence 10-20 samples or more in a single 'run' to obtain a desired level coverage (typically, 20M-40M reads on average is often sufficient for most Differential Gene Expression (DGE) experiments). 

Most of the time, the sequence that you align will ONLY be the sequence you are interested in targetting. However, you will generate read outs that will "sequence into" those adaptors. In those cases, you may need to detect the presence of these residual bits of your adaptors/libraries using computational tools (e.g., [Trimmomatic](https://pubmed.ncbi.nlm.nih.gov/24695404/)). For simplicity, we have processed the data used below (so that you don't have to run this processing step). 

In most cases of RNA-Seq, mapping is done mostly against the genome using aligners built to utilze "short" reads (150bp or less). Because reads are short - and do not capture a full transcript, this creates one additional layer of complexity - one will identify reads from varing splice isoforms that cross exon boundaries. (This is in contrast to other types of sequencing data generation, e.g. ChIP-Seq, which do not carry this complexity). 

One popular splicing aware aligner is [STAR](https://github.com/alexdobin/STAR) (Spliced Transcripts Alignment to Reference) which is considered a state-of-the-art aligner for this task.

As we're not focused on quantifying the abundance of alternatively splice transcripts, we will use alternative approaches instead, which map reads on a database of already assembled transcripts. Three leading, popular algorithms for doing this include:

[Salmon](https://combine-lab.github.io/salmon/)
<br> [kallisto](https://pachterlab.github.io/kallisto/)
<br> [RSEM](http://deweylab.github.io/RSEM/)

For this module, we will use Salmon for read mapping. Salmon take as input `.fastq` and a reference transcriptome and produces quantification files that end with `.sf`. 

A short tutorial on how to use Salmon can be found [here](https://combine-lab.github.io/salmon/getting_started/). Because of the size of the class, we can't run this alignment directly on CoCalc (because of computer memory constraints). As such, we have already provided you with Salmon output on two `.fastq` files.

As an example, we will use the airways dataset that contains RNA-Seq data of four human airway smooth muscle cell lines, n=2  treated and n=2 untreated with dexamethasone. 

Under the directory `/quants` in the Data directory that we provided you (`~/07_Data-RNASeq-I`), you will find the output of Salmon on two samples, the name of each directory corresponds to the sample name, the file `quants.sf.gz` contains the mapping information that we will load to our R workspace.

### To load Salmon mappings into R we will use the library `tximeta`, this library not only provides tool to loading and process of transcript mapping data, but also provides automatic annotation of metadata for commonly used transcriptomes.

**Q7.** Execute the code below to load the library and a table containing meta data associated with the two samples.

In [ ]:
library(tximeta)
library(SummarizedExperiment)
coldata <- read.csv("full_table.csv", row.names=1, stringsAsFactors=FALSE)
coldata

In order to load this data using `tximeta`, our `coldata` table will need to contain two additional columns: 
   
   `names`: for the name of the sample and 
   
   `files`: with the complete path to each `quants.sf.gz` file.

**Q8.** Run the code below to add these columns, take a minute to understand how the path to each file is generated. If needed, run the `file.path()` command separatly and take a look at the out it produces.

In [ ]:
coldata$names = coldata$Run
coldata$files = file.path('~/07_Data_RNASeq-I/quants', coldata$names, "quant.sf.gz")
coldata

If you wanted to run this on your own in your computational space, you could use:

`se <- tximeta(coldata)`


This would format the object and identify and download the annotation for the transcriptome index used for mapping automatically (and store in a variable called `se`).

**DO NOT RUN THIS ON COCALC.** Unfortunately, the job to generate this object is memory intensive, so not able to be run on CoCalc.

As a result, we have provided this object to you which you can load into R!

**Q9.** Execute the command below to load the data and bring in a new variable `se` into your R workspace.

In [5]:
load(file="~/07_Data_RNASeq-I/gse_se.RData")

We have just loaded a **SummarizedExperiment** (SE) object. This is an important data structure that is used as a basic structure in which RNA-Seq data is stored in R. 

You can access the  objects by their variable names `se` and `gse`.

The structure is depicted in the figure below: <br>
<img src="files/SEstructure.png" width="400" height="200">

`coldata` conatins the data about the samples which we have provided to the tximeta function.

`rowRanges` contains information about each gene/transcripts, this data was downloaded automatically because we used a standard transcriptome assembly. 

`assays` is part of the SE object and contains the actual counts. Note that there can be more than one assay associated with different processing levels, e.g. normalized and non-normalized counts.

### Please review sections one and two of this [vignette](https://bioconductor.org/packages/release/bioc/vignettes/SummarizedExperiment/inst/doc/SummarizedExperiment.html) to familiarize yourself with the structure of the SummarizedExperiment object.

**Q10.** Now:

- Use `dim()` to obtain the dimensions of your current SE object and report how many transcripts are quantified.
- Use `head()` to get a summary of `se`.

**Provide and Execute your code below.**

In [ ]:
dim(se)
head(se)

**Q11.** You will notice that there are three assays, use the code below to list their names:

In [ ]:
assayNames(se)

The three assays correspond to the output from the Salmon aligner:

`counts` is the non-normalized read count per transcript.

`abundance` is the estimated relative abundance of this transcript in units of TPM (transcripts per million).

`length` is the length of the transcript (effective length, see below).

**Q12.** Take a look at these assays using the command 

`head(assays(se)$count)`

Next,

- perform the above with `abundance` and `length`.
- summarize the information about each transcript using `rowRanges(se)`

Note that transcript length is not same for the two samples!

The reason for this is that it is harder to capture the ends of the transcripts, and the Salmon algorithm estimates an *effective* length for each sample separately.

In [2]:
#explore head(assays(se)$count)


In [ ]:
#Perform with abundance and length


In [ ]:
#summarize using rowRanges(se)


Next, because we are interested in gene and not transcript information, we will need to summarize transcript data to genes.

If you wanted to run this on your own in your computational space, you could use:

`gse <- summarizeToGene(se)`

This would format the object and identify and download the annotation for the transcriptome index used for mapping automatically (and store in a variable called `gse`).

**DO NOT RUN THIS ON COCALC.** Unfortunately, the job to generate this object is memory intensive, so not able to be run on CoCalc.



As a result, we have also provided you this object (which you already loaded into R in **Q9**) called `gse`!

**Q13.** Now:

- Use `dim()` to obtain the dimensions of the `gse` object and report how many genes are quantified.
- Use `head()` to get a summary of `gse`

Note that the dimentions are now much smaller because transcript isoforms have been aggregated to single genes.

There are some addtional functions that can be used to explore the gene "metadata".

**Q14.** Run the code below to access gene sequence information:

In [ ]:
rowRanges(gse)
seqinfo(rowRanges(gse))

Our next step(s) will be to:

- Normalize the data
- Perform a statistical test to look for genes that are differently expressed between treatment and controls

This will be the focus of the next session. 

To perform statistical testing, we do need to consider how the data is distributed in order to select the proper statistical test to measure expression differences between treatment and control. 

For example, we could use a t-test if expression levels are approximately normally distributed. 

**Q15.** Create a histogram of the count assay matrix, use a log10 scale. 

**Provide and Execute your code below.**

This is not *quite* a normal distribution. 

It turns out that gene expression count data is [better](http://bridgeslab.sph.umich.edu/posts/why-do-we-use-the-negative-binomial-distribution-for-rnaseq) [modeled](https://bioramble.wordpress.com/2016/01/30/why-sequencing-data-is-modeled-as-negative-binomial/) using the [Negative Binomial Distribution](https://stattrek.com/probability-distributions/negative-binomial.aspx). 

We will cover this in the next session!